In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
torch.cuda.is_available()

True

In [4]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [36]:
#model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128,625,bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625,10,bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0),-1)
        # print(out.shape)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [37]:
model = CNN().to(device)
value = torch.Tensor(1,1,28,28).to(device)
print(model(value).shape)

torch.Size([1, 10])


In [38]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [39]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost=0
    
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        
        prediction = model(X)
        cost = criterion(prediction,y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print(f'Epoch {epoch}, Cost = {avg_cost}')
    
print('learning finish')

Epoch 0, Cost = 0.16081148386001587
Epoch 1, Cost = 0.04368942603468895
Epoch 2, Cost = 0.029210302978754044
Epoch 3, Cost = 0.021842025220394135
Epoch 4, Cost = 0.018934298306703568
Epoch 5, Cost = 0.014823882840573788
Epoch 6, Cost = 0.013132160529494286
Epoch 7, Cost = 0.01145907398313284
Epoch 8, Cost = 0.008581727743148804
Epoch 9, Cost = 0.008708802051842213
Epoch 10, Cost = 0.007401136681437492
Epoch 11, Cost = 0.005385441705584526
Epoch 12, Cost = 0.005866860039532185
Epoch 13, Cost = 0.007055776193737984
Epoch 14, Cost = 0.004889339208602905
learning finish


In [40]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_predictoin = torch.argmax(prediction,1)== y_test
  acc = correct_predictoin.float().mean()
  print('acc:', acc.item())

acc: 0.9811999797821045


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
